In [ ]:
# 최근 공시보고서 텔레그램 메시지 자동 알림


import time # sleep을 주기 위해 사용하는 모듈
import telegram

# key
API_TOKEN = '872703060:AAHZw-LOLO8qH2OIqL_jCtVvVo28-bQEO9k'
chat_id =  "868118183"

#텔레그램 봇
bot = telegram.Bot(token=API_TOKEN)




def get_disclosure_info(addr):  
    
    #전역변수
    global disc_dict    
    

    # 최근공시페이지 정보 가져오기 : get요청
    import requests
    req = requests.get(addr)  
    
    # 페이지 정보 문자열로 변환
    html = req.text      
    
    # BeautifulSoup 객체를 생성
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    
    # 최근공시목록 파싱
    test = soup.find_all('tr')
    
    
    # 최근공시 10개를 담을 딕셔너리 생성
    
    disc_dict = { '제출시간' : [] , '접수번호':[], '회사명': [] ,'보고서명':[],'제출자':[],'제출일자':[],'보고서링크':[] }
    
    
    
    # 최근공시목록 10건에서 필요한 정보만을 파싱하는 작업   
        
    for i in test[1:100]:

        
        result = i.find_all('td')
        
        ### 제출시간
        sub_time = result[0].text.strip()
        
        ### 회사명
        corp_name = result[1].text.strip()
        
        ### 보고서명
        report_name = result[2].text.strip()
        
        ### 제출자
        submitter = result[3].text.strip()
        
        ### 제출일자
        submit_day = result[4].text.strip()
        
        ## 접수번호 추출(고유번호)
        addr = 'http://dart.fss.or.kr/' + result[2].a['href'] 
        
        import re
        stime_pattern = re.compile('(=\d+$)')
        rcp_no = stime_pattern.search(addr)[0][1:]

        
        # 필요한 정보 딕셔너리에 저장
        
        disc_dict['제출시간'].append(sub_time)
        disc_dict['접수번호'].append(rcp_no)
        disc_dict['회사명'].append(corp_name)
        disc_dict['보고서명'].append(report_name)
        disc_dict['제출자'].append(submitter)
        disc_dict['제출일자'].append(submit_day)

    



    
    

        
# 텔레그램 실시간 알림


#switch
switch = False


while True:
    
    import requests
    
    
    # get_disclosure_info함수를 사용하여 최근공시 10건을 담은 딕셔너리 생성
    get_disclosure_info('http://dart.fss.or.kr/dsac001/mainAll.do')

    
    
    if switch == False:
        
        #  실시간 첫알림 최근공시 1개만 push
        
        before = disc_dict['접수번호'][0]

        link = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + before

        chat_text = "[%s] %s , %s \n %s \n %s" % (disc_dict['회사명'][0],disc_dict['제출일자'][0],disc_dict['제출시간'][0],
                                                      disc_dict['보고서명'][0],link )

        url_tmpl = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s"
        url =  url_tmpl % (API_TOKEN, chat_id, chat_text)


        r = requests.get(url)

        
        switch =True
    
    
    
    
    
    
    else:
        

        latest = disc_dict['접수번호'][0]
                 
       
        if before != latest:           # 이전 루프의 최근공시 접수번호(before) 와 일치하는지 확인    
            
            idx = disc_dict['접수번호'].index(before)
                        
            
            for i in reversed(range(idx)):   # 1분동안 새롭게 공시된 정보를 업로드 순서대로 알림 push 
                               
                link = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + disc_dict['접수번호'][i] 

                chat_text = "[%s] %s , %s \n %s \n %s" % (disc_dict['회사명'][i],disc_dict['제출일자'][i],disc_dict['제출시간'][i],
                                                              disc_dict['보고서명'][i],link)

                url_tmpl = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s"
                url =  url_tmpl % (API_TOKEN, chat_id, chat_text)

                
                r = requests.get(url)
                
                
                
                
                

        #테스트 목적
        else:
            
            
            bot.sendMessage(chat_id=chat_id , text='새로운 공시가 없습니다.')   


            
        
        #가장 최근공시 접수번호(lates)를 다음 루프에 사용될 before 변수에 저장
        before = latest
        
        
       
    
    
    
    
        
    time.sleep(60)









In [4]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

# 실제 인증키를 사용

auth = 'ee4a8a2afef9bc27346e66698976c9da023296b5' 


# 최근 공시보고서 텔레그램 메시지 자동 알림



# key
API_TOKEN = '872703060:AAHZw-LOLO8qH2OIqL_jCtVvVo28-bQEO9k'
chat_id = "868118183"

#텔레그램 봇
bot = telegram.Bot(token=API_TOKEN)


#switch
switch = False




def mapping_func(data,col_name):
    
    #기타 정보 상세화
    
    corp_type = {'Y':'유가증권시장',
            'K':'코스닥',
            'N':'코넥스',
            'E':'기타'}


    dept ={ '유':'KRX 코스닥시장본부 소관',
            '코':'KRX 코스닥시장본부 소관',
            '채':'KRX 채권상장법인 공시사항',
            '넥':'KRX 코넥스시장 소관',
            '공':'공정거래위원회 소관',
            '연':'연결부분포함',
            '정':'제출후정정신고존재',
            '철':'철회(간주)'}
    
    
    
    #해당 정보추출작업
    
    if col_name == '법인구분' or    col_name == '비고':    
    
        try:           
            
            if col_name == '법인구분':
            
                return corp_type[data[col_name].values[0]]
    
    
            else:

                return dept[data[col_name].values[0]]
        
        
        except: # col_name 이 없는 경우
            
            return ''
                
                
    else:
        
        return data[col_name].values[0]
        
 




        
#automation


# url_tmpl = f'http://dart.fss.or.kr/api/search.json?auth={auth}'

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)
r = requests.get(url)
jo = json.loads(r.text)

result = json_normalize(jo, 'list')


cols = {'crp_cd':'종목코드','crp_cls':'법인구분','crp_nm':'종목명','flr_nm':'제출인',
    'rcp_dt':'접수날짜','rcp_no':'접수번호','rmk':'비고','rpt_nm':'보고서명'}

result.rename(columns=cols, inplace=True)


print(result)

# if switch == False:



#     recent_disc =[]
#     recent_disc.append(result['접수번호'][0])
#     before = recent_disc[0]             

#     new_disc = result.loc[result['접수번호'] == before,:]


#     link = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + before
#     chat_text = "[%s] %s \n %s \n %s \n %s" % ( mapping_func(new_disc,'종목명'),mapping_func(new_disc,'법인구분'),
#                                                 mapping_func(new_disc,'보고서명'),mapping_func(new_disc,'비고'),link )

#     url_tmpl = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s"
#     url =  url_tmpl % (API_TOKEN, chat_id, chat_text)


#     r = requests.get(url)


#     switch =True






# else:



#     recent_disc =  list(result['접수번호'])

#     recent_disc_arr = np.array(recent_disc)
#     latest = recent_disc_arr[0]




#     if before != latest:   #error


#         idx = np.where(recent_disc_arr == before)[0][0]



#         for i in recent_disc_arr[:idx][::-1]:

#             #공시알림
#             new_disc = result.loc[result['접수번호'] == i,:]



#             link = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + i
#             chat_text = "[%s] %s \n %s \n %s \n %s" % ( mapping_func(new_disc,'종목명'),mapping_func(new_disc,'법인구분'),
#                                                         mapping_func(new_disc,'보고서명'),mapping_func(new_disc,'비고'),link )

#             url_tmpl = "https://api.telegram.org/bot%s/sendMessage?chat_id=%s&text=%s"
#             url =  url_tmpl % (API_TOKEN, chat_id, chat_text)


#             r = requests.get(url)




#     #테스트 목적
#     else:


#         bot.sendMessage(chat_id=chat_id , text='새로운 공시가 없습니다.')   





#     before = latest


       
    
    
    
    
        










Empty DataFrame
Columns: []
Index: []
